## 使用 LLM 進行文本摘要

由於 LLM「理解」一種語言，因此它適合拿來做翻譯或摘要等等任務。

而本 Notebook 中，將使用 LLM 來總結一些文本內容，特別是索賠範例內容。

### Requirements and Imports

如果有依照前面流程啟用 Workbench Image 的話，預設都會包含所需 Libraries，因此可以直接 Import 使用。但如果未使用正確 Image 的話，則需要執行第一行 `pip install ...` 來安裝相依套件。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchain pipeline

透過 Langchain 定義 summarization pipeline。並使用已經部署好的 **Mistral-7B Instruct v2** 來進行內容摘要產生。

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:3000/"

# LLM definition
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

建立一個 **template** 讓 LLM 模型針對特定內容摘要任務進行格式化。例如下面 Template 最後一段定義:

`我會給你(模型)一篇文字，你(模型)必須盡可能地總結內容給我`

In [ ]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant.
Always assist with care, respect, and truth. Respond with utmost utility yet securely.
Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
I will give you a text that you must summarize as best as you can.

### TEXT:
{input}

### SUMMARY:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

Langchain 現在允許將這些元素「縫合」在一起，並建立一個用來查詢模型的 **conversation** 物件。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

現在已經準備完成查詢模型了！

此時可以從 `claims` 目錄中，使用一些一些 JSON 格式的範例內容來驗證。下面程式將讀取檔案內容，然後讓 LLM 幫我們產生摘要內容。

In [ ]:
# Read the claims and populate a dictionary
claims_path = 'claims'
onlyfiles = [f for f in listdir(claims_path) if isfile(join(claims_path, f))]

claims = {}

for filename in onlyfiles:
    # Opening JSON file
    with open(os.path.join(claims_path, filename), 'r') as file:
        data = json.load(file)
    claims[filename] = data

In [ ]:
for filename in onlyfiles:
    print(f"***************************")
    print(f"* Claim: {filename}")
    print(f"***************************")
    print("Original content:")
    print("-----------------")
    print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
    print('Summary:')
    print("--------")
    summary_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
    conversation.predict(input=summary_input);
    print("\n\n                          ----====----\n")